In [2]:
import numpy as np
import pandas as pd

import nationality_helpers
from nationality_helpers import create_top_medalist

In [3]:
df = create_top_medalist()

In [4]:
df.shape

(1072, 13)

### To do:

### Encode nationality, sex, sport.  DOB to datetime.  Split into male/female.  Then split athletics vs not athletics.  Athletics contains all of track.  This is a large range from distance runners to shot put throwers.

In [5]:
df.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze,medal_or_nm,country_count
0,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1,1,69
1,769580282,Akeem Haynes,CAN,male,3/11/92,1.68,71.0,athletics,0,0,1,1,69
2,373002185,Allison Beveridge,CAN,female,6/1/93,1.69,62.0,cycling,0,0,1,1,69
3,686662012,Allysha Chapman,CAN,female,1/25/89,1.60,58.0,football,0,0,1,1,69
4,857846421,Andre de Grasse,CAN,male,11/10/94,1.76,70.0,athletics,0,1,2,3,69


In [6]:
df.dtypes

id                 int64
name              object
nationality       object
sex               object
dob               object
height           float64
weight           float64
sport             object
gold               int64
silver             int64
bronze             int64
medal_or_nm        int64
country_count      int64
dtype: object

In [7]:
df.groupby('sport')['country_count'].agg('count')

sport
aquatics             221
archery               11
athletics             83
badminton              8
basketball            48
canoe                 27
cycling               46
equestrian            39
fencing               40
football              54
golf                   3
gymnastics            50
handball              60
hockey                49
judo                  28
modern pentathlon      3
rowing                79
rugby sevens          36
sailing               20
shooting              28
table tennis          18
taekwondo              9
tennis                 9
triathlon              4
volleyball            65
weightlifting          9
wrestling             25
Name: country_count, dtype: int64

In [9]:
df.describe()

,id,height,weight,gold,silver,bronze,medal_or_nm,country_count
count,1.072000e+03,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000
mean,4.974190e+08,1.788759,73.985075,0.400187,0.347948,0.375000,1.123134,134.834888
std,2.860416e+08,0.120626,15.734366,0.579223,0.508761,0.503257,0.446740,67.629129
min,2.938886e+06,1.400000,33.000000,0.000000,0.000000,0.000000,1.000000,53.000000
25%,2.510752e+08,1.700000,63.000000,0.000000,0.000000,0.000000,1.000000,82.000000
50%,4.973888e+08,1.780000,72.000000,0.000000,0.000000,0.000000,1.000000,109.000000
75%,7.460005e+08,1.880000,83.000000,1.000000,1.000000,1.000000,1.000000,159.000000
max,9.993741e+08,2.210000,143.000000,5.000000,2.000000,2.000000,6.000000,258.000000


### Checking for data integrity issues.  Height and weight seem feasible.